<a href="https://colab.research.google.com/github/beyzaatasever/als-movie-recommender/blob/main/collabrative_filtering_ipynb_adl%C4%B1_dosyan%C4%B1n_kopyas%C4%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd


movies_df = pd.read_csv('movies.csv')
ratings_df = pd.read_csv('ratings.csv')


print(movies_df.head())



   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


In [15]:
import pandas as pd


films_df = pd.read_csv('films.csv')


print(films_df.head())
films_df.head()


         Date                 Name  Year        Letterboxd URI
0  2023-01-25            Gone Girl  2014  https://boxd.it/6hQu
1  2023-02-08  The Virgin Suicides  1999  https://boxd.it/28JG
2  2023-02-13   Dancer in the Dark  2000   https://boxd.it/71E
3  2023-02-18    Girl, Interrupted  1999  https://boxd.it/25ds
4  2023-02-26           Lux Æterna  2019  https://boxd.it/m6b8


,Date,Name,Year,Letterboxd URI
0,2023-01-25,Gone Girl,2014,https://boxd.it/6hQu
1,2023-02-08,The Virgin Suicides,1999,https://boxd.it/28JG
2,2023-02-13,Dancer in the Dark,2000,https://boxd.it/71E
3,2023-02-18,"Girl, Interrupted",1999,https://boxd.it/25ds
4,2023-02-26,Lux Æterna,2019,https://boxd.it/m6b8


In [16]:

print("Movies DataFrame:")
print(movies_df.head())

print("\nRatings DataFrame:")
print(ratings_df.head())

print("\nFilms DataFrame:")
print(films_df.head())

Movies DataFrame:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  

Ratings DataFrame:
   userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931

Films DataFrame:
         Date                 Name  Year        Letterboxd URI
0  2023-01-25            Gone Girl  2014  https://boxd.it/

In [17]:
# extract year column
import pandas as pd
import re  # 're' modülünü içe aktarıyoruz
def extract_year(title):
    match = re.search(r'\((\d{4})\)', title)
    if match:
        return int(match.group(1))
    return None

movies_df['year'] = movies_df['title'].apply(extract_year)

# check year column
print("Movies DataFrame with extracted year:")
print(movies_df.head())

movies_df = movies_df.dropna(subset=['year']) #na leri kaldırdım
#delete NaN values
movies_df['year'] = movies_df['year'].astype(int)

# check new year column
print("Movies DataFrame with extracted year:")
print(movies_df.head())

Movies DataFrame with extracted year:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres    year  
0  Adventure|Animation|Children|Comedy|Fantasy  1995.0  
1                   Adventure|Children|Fantasy  1995.0  
2                               Comedy|Romance  1995.0  
3                         Comedy|Drama|Romance  1995.0  
4                                       Comedy  1995.0  
Movies DataFrame with extracted year:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride

<ipython-input-17-16c0c0dcff6e>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_df['year'] = movies_df['year'].astype(int)


In [18]:

print("Movies DataFrame:")
print(movies_df.head())

print("\nFilms DataFrame:")
print(films_df.head())

Movies DataFrame:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  year  
0  Adventure|Animation|Children|Comedy|Fantasy  1995  
1                   Adventure|Children|Fantasy  1995  
2                               Comedy|Romance  1995  
3                         Comedy|Drama|Romance  1995  
4                                       Comedy  1995  

Films DataFrame:
         Date                 Name  Year        Letterboxd URI
0  2023-01-25            Gone Girl  2014  https://boxd.it/6hQu
1  2023-02-08  The Virgin Suicides  1999  https://boxd.it/28JG
2  2023-02-13   Dancer in the Dark  2000   https://boxd.it/71E
3  2023-02-18    Girl, Interrupted  1999  https://boxd.it/25ds
4  2023-02-26     

In [19]:

films_df['Name'] = films_df['Name'].str.lower().str.strip()
movies_df['title'] = movies_df['title'].str.lower().str.strip()

print("Cleaned Movies DataFrame with integer year:")
print(movies_df.head())
print("\nFilms DataFrame:")
print(films_df.head())


Cleaned Movies DataFrame with integer year:
   movieId                               title  \
0        1                    toy story (1995)   
1        2                      jumanji (1995)   
2        3             grumpier old men (1995)   
3        4            waiting to exhale (1995)   
4        5  father of the bride part ii (1995)   

                                        genres  year  
0  Adventure|Animation|Children|Comedy|Fantasy  1995  
1                   Adventure|Children|Fantasy  1995  
2                               Comedy|Romance  1995  
3                         Comedy|Drama|Romance  1995  
4                                       Comedy  1995  

Films DataFrame:
         Date                 Name  Year        Letterboxd URI
0  2023-01-25            gone girl  2014  https://boxd.it/6hQu
1  2023-02-08  the virgin suicides  1999  https://boxd.it/28JG
2  2023-02-13   dancer in the dark  2000   https://boxd.it/71E
3  2023-02-18    girl, interrupted  1999  https://boxd.

In [20]:
# merged films
merged_df = pd.merge(films_df, movies_df, left_on=['Name', 'Year'], right_on=['title', 'year'], how='inner')

In [21]:
# check frames
print("Films DataFrame:")
print(films_df[['Name', 'Year']].head())
print(films_df.dtypes)

print("\nMovies DataFrame:")
print(movies_df[['title', 'year']].head())
print(movies_df.dtypes)


Films DataFrame:
                  Name  Year
0            gone girl  2014
1  the virgin suicides  1999
2   dancer in the dark  2000
3    girl, interrupted  1999
4           lux æterna  2019
Date              object
Name              object
Year               int64
Letterboxd URI    object
dtype: object

Movies DataFrame:
                                title  year
0                    toy story (1995)  1995
1                      jumanji (1995)  1995
2             grumpier old men (1995)  1995
3            waiting to exhale (1995)  1995
4  father of the bride part ii (1995)  1995
movieId     int64
title      object
genres     object
year        int64
dtype: object


In [22]:

films_selected = films_df[['Name', 'Year']]

print("Selected Films DataFrame:")
print(films_selected)

films_list = films_selected.values.tolist()


print("Films List:")
print(films_list)


Selected Films DataFrame:
                               Name  Year
0                         gone girl  2014
1               the virgin suicides  1999
2                dancer in the dark  2000
3                 girl, interrupted  1999
4                        lux æterna  2019
5                      irreversible  2002
6             2001: a space odyssey  1968
7                            amélie  2001
8                           persona  1966
9        portrait of a lady on fire  2019
10    the worst person in the world  2021
11                    normal people  2020
12                       frances ha  2012
13        how to save a dead friend  2022
14                       possession  1981
15       10 things i hate about you  1999
16                        manifesto  2015
17  the hunger games: catching fire  2013
18                     bye bye body  2019
19                        sidewalls  2011
20                 marie antoinette  2006
21                 paris is burning  1990
22      

In [23]:
# found matched films
def find_matching_movie(name, year, movies_df):
    matching_movies = movies_df[(movies_df['year'] == year) & (movies_df['title'].str.contains(name, case=False, na=False))]
    return matching_movies

# found for al films matched
matched_movies = []
for name, year in films_list:
    matched = find_matching_movie(name, year, movies_df)
    if not matched.empty:
        matched_movies.append(matched)

# Turned matched films into dataframes
if matched_movies:
    matched_movies_df = pd.concat(matched_movies)
else:
    matched_movies_df = pd.DataFrame()

print("\nMatched Movies DataFrame:")
print(matched_movies_df)



Matched Movies DataFrame:
      movieId                                              title  \
8467   112556                                   gone girl (2014)   
2915     3910                          dancer in the dark (2000)   
2402     3186                           girl, interrupted (1999)   
4258     6214                 irreversible (irréversible) (2002)   
706       924                       2001: a space odyssey (1968)   
3622     4973  amelie (fabuleux destin d'amélie poulain, le) ...   
4893     7327                                     persona (1966)   
8173   102800                                  frances ha (2012)   
1940     2572                  10 things i hate about you (1999)   
8295   106487             the hunger games: catching fire (2013)   
7954    96020                      sidewalls (medianeras) (2011)   
6226    45880                            marie antoinette (2006)   
895      1192                            paris is burning (1990)   
4615     6874        

In [24]:
# lets give new ID to the user
user_id = 9999

# adding ratings for user
user_ratings = matched_movies_df[['movieId']].copy()
user_ratings['userId'] = user_id
user_ratings['rating'] = 5  # added 5 stars

# adding new rating to old ones
all_ratings = pd.concat([ratings_df[['userId', 'movieId', 'rating']], user_ratings])

print("\nAll Ratings DataFrame (Yeni Kullanıcı Ratingleriyle):")
print(all_ratings.tail())



All Ratings DataFrame (Yeni Kullanıcı Ratingleriyle):
      userId  movieId  rating
8295    9999   106487     5.0
7954    9999    96020     5.0
6226    9999    45880     5.0
895     9999     1192     5.0
4615    9999     6874     5.0


In [28]:


from pyspark.sql import SparkSession
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

spark = SparkSession.builder.appName("movie-recommendation").getOrCreate()

all_ratings_spark = spark.createDataFrame(all_ratings)


(training, test) = all_ratings_spark.randomSplit([0.8, 0.2])


als = ALS(
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    nonnegative=True,
    implicitPrefs=False,
    coldStartStrategy="drop",
    rank=10,
    maxIter=15,
    regParam=0.1
)


model = als.fit(training)


predictions = model.transform(test)

# Tahminleri ve gerçek değerleri göster
predictions.show()

# RMSE'yi hesaplayalım
evaluator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)

rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))


# Calculating MAE
evaluator_mae = RegressionEvaluator(
    metricName="mae",
    labelCol="rating",
    predictionCol="prediction"
)

mae = evaluator_mae.evaluate(predictions)
print("Mean Absolute Error = " + str(mae))


# Make recomandations
user_subset = all_ratings_spark.filter(all_ratings_spark.userId == user_id)
recommendations = model.recommendForUserSubset(user_subset, numItems=10)

recommendations_df = recommendations.toPandas()
recommended_movie_ids = [row.movieId for row in recommendations_df.iloc[0]['recommendations']]


recommended_movies = movies_df[movies_df['movieId'].isin(recommended_movie_ids)]
print("\nRecommended Movies:")
print(recommended_movies[['movieId', 'title']])


+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   148|    356|   4.0| 3.5306149|
|   148|  31658|   4.0| 3.3682349|
|   148|  40815|   4.0| 3.6670384|
|   148|  44191|   4.0| 3.2336102|
|   148|  50872|   3.0| 3.9359417|
|   148|  68954|   4.0| 3.7163663|
|   148|  76093|   3.0| 4.1753197|
|   148|  79091|   3.5|  3.473277|
|   148|  88125|   4.0| 3.8257494|
|   243|     36|   4.0| 3.4285364|
|   243|    193|   5.0| 2.9932742|
|   243|    420|   4.0| 4.1511383|
|   243|    589|   4.0| 4.9519553|
|    31|   1240|   4.0|   4.36797|
|    31|   1265|   2.0|    4.3014|
|    85|   1358|   1.0| 4.3890443|
|    85|   1446|   4.0| 3.8253639|
|    85|   1635|   4.0|  4.203715|
|    85|   1683|   3.0| 3.1473725|
|    85|   1694|   4.0|  3.030255|
+------+-------+------+----------+
only showing top 20 rows

Root-mean-square error = 0.8795960814832753
Mean Absolute Error = 0.6805277827791986

Recommended Movies:
      movieId               